<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from skimage.color import rgb2gray
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from skimage import io

Using TensorFlow backend.


In [3]:
def TF(X_train,y_train):

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(105, 105, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(20))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size=250,epochs=5,verbose=1,validation_split=0.2)
    
    return model

# Load training data 
training_data_path = Path("cleaned_train.pkl") 
data_from_nb1 = pd.read_pickle(training_data_path)

# Tranform into grayscale(as not all images are coloured) and convert it into same size
nn_input = data_from_nb1.apply(lambda x: resize(rgb2gray(x['image']), (105, 105)), axis=1)
X = np.array([list(i.reshape(105,105,1)) for i in nn_input])
y = pd.get_dummies(data_from_nb1['encoding'])

#Split into train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state = 42)

#Fit the model
model_nn = TF(X_train,y_train)

scores = model_nn.evaluate(X_val, y_val)
print('Accuracy on test data: ', scores[1])

,name,image,category,encoding
0,airplanes/airplanes_0001.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",airplanes,0
1,airplanes/airplanes_0002.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",airplanes,0
2,airplanes/airplanes_0003.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",airplanes,0
3,airplanes/airplanes_0004.jpg,"[[[174, 190, 187], [173, 189, 186], [172, 188,...",airplanes,0
4,airplanes/airplanes_0005.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",airplanes,0


In [5]:
# Train final model on whole training data set 
model_nn = TF(X, y) 

# Load test data 
test_path = '20_Validation/' # Wrong naming in project should be 20_Test 
file_list= os.listdir(test_path)
images = [io.imread(test_path + animal) 
              for animal in file_list 
              if os.path.isfile(test_path + animal)]
test_df = pd.DataFrame(images, columns=['image'])

# Tranform input
test_df = test_df.apply(lambda x: resize(rgb2gray(x['image']), (105, 105)), axis=1)
X_test = np.array([list(i.reshape(105,105,1)) for i in test_df])

# Predict using model
predictions = model_nn.predict(X_test)

# Outputting the to a csv
output_file_name = 'test_results_neural_network'
pd.DataFrame(predictions).apply(lambda x: np.argmax(x), axis=1).to_csv(output_file_name)
print('Saved output to: ', output_file_name)

,image
0,"[[[48, 53, 57], [52, 57, 63], [54, 59, 65], [4..."
1,"[[[213, 173, 85], [213, 173, 85], [213, 173, 8..."
2,"[[[19, 36, 30], [14, 31, 25], [11, 26, 21], [1..."
3,"[[[95, 138, 69], [95, 138, 69], [95, 138, 69],..."
4,"[[[162, 204, 242], [164, 203, 242], [164, 203,..."
